In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/Users/kentbaluyot/Desktop/Labor Force Survey/LFS_march2324.csv", low_memory=False)

In [3]:
pd.set_option("display.max_columns", None)
df.head()

,RDMD_ID,Region,Household Unique Sequential Number,Final Weight Based on Projection (provincial projections),Final Weight Based on Projection (provincial projections).1,Survey Month,Survey Year,Psu Number,Replicate,Household Size,C101-Line Number,C03-Relationship to Household Head,C04-Sex,C05-Age as of Last Birthday,C06-Marital Status,C07-Highest Grade Completed,C08-Overseas Filipino Indicator,C09-Work Indicator,C09A-Work Indicator,C09A-Work Indicator.1,C10-Job Indicator,"C11 - Location of Work (Province, Municipality)",C13-Major Occupation Group,C15-Major Industry Group,C16-Nature of Employment (Primary Occupation),C17-Normal Working Hours per Day,C18-Total Number of Hours Worked during the past week,C19-Want More Hours of Work,C20-Look for Additional Work,C20B - First time to do any work,C21-Class of Worker (Primary Occupation),C22-Other Job Indicator,C23-Total Hours Worked for all Jobs,C24-Reasons for Working More than 48 Hours during the past week,C25-Looked for Work or Tried to Establish Business during the past week,C25B - First time to look for work,C26-Reason for not Looking for Work,C27-Available for Work,C28-Previous Job Indicator,C29 - Last worked (Year),C29 - Last worked (Month),C31-Previous Occupation,C33-Kind of Business (past quarter),"New Employment Criteria (jul 05, 2005)",C08-Currently Attending School,C09-Graduate of technical/vocational course,C09a - Currently Attending Non-formal Training for Skills Development,C10-Overseas Filipino Indicator,C11-Work Indicator,C11A - Working Arrangement,C12-Job Indicator,"C12A - Location of Work (Province, Municipality)",C14-Primary Occupation,C16-Kind of Business (Primary Occupation),C17-Nature of Employment (Primary Occupation),C18-Normal Working Hours per Day,C19-Total Number of Hours Worked during the past week,PUFC19B_PWHRS,C20-Want More Hours of Work,C21-Look for Additional Work,C22-First Time to Work,C23-Class of Worker (Primary Occupation),C24-Basis of Payment (Primary Occupation),C25-Basic Pay per Day (Primary Occupation),C26-Other Job Indicator,C27-Number of Jobs during the past week,C28-Total Hours Worked for all Jobs,C29-Reasons for Working More than 48 Hours during the past week,C30-Looked for Work or Tried to Establish Business during the past week,C31-First Time to Look for Work,C32-Job Search Method,C33-Number of Weeks Spent in Looking for Work,C34-Reason for not Looking for Work,C35-When Last Looked for Work,C36-Available for Work,C37-Willingness to take up work during the past week or withing two weeks,C38-Previous Job Indicator,C39 - Last worked (Year),C39 - Last worked (Month),C41- Previous Occupation,C43-Kind of Business (past quarter),2015Urban-RuralFIES,PUFC05B_ETHNICITY
0,1,1,1,1511.3933,3,2024,636,30,5,1,1,1,41,2,50611,5,1,1,,2805,61,01,1,02,014,2,2,2,3,2,014,20,,,,,,,,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,1520.9426,3,2024,636,30,5,2,2,2,39,2,40913,1,,,,,,,,,,,,,,,,,,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,1536.8579,3,2024,636,30,5,3,3,2,18,1,34013,5,2,,2,,,,,,,,,,,,,,2,,08,,2,,,,,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1,1162.8655,3,2024,636,30,5,4,3,1,15,1,24013,5,2,,2,,,,,,,,,,,,,,2,,08,,2,,,,,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,1,1508.0276,3,2024,636,30,5,5,8,1,62,1,24015,5,1,1,,2805,61,01,1,05,026,2,2,2,4,2,026,20,,,,,,,,,,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
updated_columns = ['Region',
                   'C09A-Work Indicator', 'C09A-Work Indicator.1',
                   'C11 - Location of Work (Province, Municipality)', 'C13-Major Occupation Group'] 
df_updated = df[updated_columns]
df_updated.columns = ['Region',
                      'C10-Job Indicator', 'C11 - Location of Work (Province, Municipality)',
                      'C15-Major Industry Group', 'C16-Nature of Employment (Primary Occupation)']

In [5]:
df_updated.isnull().sum()

Region                                                   0
C10-Job Indicator                                  1787145
C11 - Location of Work (Province, Municipality)    1787145
C15-Major Industry Group                           1787145
C16-Nature of Employment (Primary Occupation)      1787145
dtype: int64

In [6]:
df_cleaned = df_updated.dropna().reset_index(drop=True)

In [7]:
df_cleaned

,Region,C10-Job Indicator,"C11 - Location of Work (Province, Municipality)",C15-Major Industry Group,C16-Nature of Employment (Primary Occupation)
0,1,,2805,01,1
1,1,,,,
2,1,2,,,
3,1,2,,,
4,1,,2805,01,1
...,...,...,...,...,...
404841,19,,4712,47,
404842,19,2,,,
404843,19,2,,,
404844,19,,,,


In [8]:
df_cleaned.dtypes

Region                                              int64
C10-Job Indicator                                  object
C11 - Location of Work (Province, Municipality)    object
C15-Major Industry Group                           object
C16-Nature of Employment (Primary Occupation)      object
dtype: object

In [9]:
# Get object columns
object_cols = df_cleaned.select_dtypes(include=['object']).columns

# Step 1: Strip all whitespace and replace any remaining empty with '0'
for col in object_cols:
    df_cleaned[col] = df_cleaned[col].astype(str).str.strip()  # Remove leading/trailing spaces
    df_cleaned[col] = df_cleaned[col].replace('', '0')  # Replace empty strings with '0'

# Step 2: Convert to int64
df_cleaned[object_cols] = df_cleaned[object_cols].astype('int64')

In [10]:
df_cleaned

,Region,C10-Job Indicator,"C11 - Location of Work (Province, Municipality)",C15-Major Industry Group,C16-Nature of Employment (Primary Occupation)
0,1,0,2805,1,1
1,1,0,0,0,0
2,1,2,0,0,0
3,1,2,0,0,0
4,1,0,2805,1,1
...,...,...,...,...,...
404841,19,0,4712,47,0
404842,19,2,0,0,0
404843,19,2,0,0,0
404844,19,0,0,0,0


In [11]:
df_cleaned.loc[
    (df_cleaned['C10-Job Indicator'] >= 0) & 
    (df_cleaned['C11 - Location of Work (Province, Municipality)'] > 0), 
    'C10-Job Indicator'
] = 1

df_cleaned.loc[
    (df_cleaned['C10-Job Indicator'] >= 0) & 
    (df_cleaned['C11 - Location of Work (Province, Municipality)'] == 0), 
    'C10-Job Indicator'
] = 2

df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

,Region,C10-Job Indicator,"C11 - Location of Work (Province, Municipality)",C15-Major Industry Group,C16-Nature of Employment (Primary Occupation)
0,1,1,2805,1,1
1,1,2,0,0,0
2,1,2,0,0,0
3,1,2,0,0,0
4,1,1,2805,1,1
...,...,...,...,...,...
404841,19,1,4712,47,0
404842,19,2,0,0,0
404843,19,2,0,0,0
404844,19,2,0,0,0


In [12]:
cols = [
    'Region',
    'C10-Job Indicator', 
    'C11 - Location of Work (Province, Municipality)',
    'C15-Major Industry Group',
    'C16-Nature of Employment (Primary Occupation)'
]

df_cleaned[cols] = df_cleaned[cols].astype('object')
df_cleaned.dtypes

Region                                             object
C10-Job Indicator                                  object
C11 - Location of Work (Province, Municipality)    object
C15-Major Industry Group                           object
C16-Nature of Employment (Primary Occupation)      object
dtype: object

In [13]:
df_cleaned = df_cleaned[df_cleaned['Region'] != 15].reset_index(drop=True)
df_cleaned

,Region,C10-Job Indicator,"C11 - Location of Work (Province, Municipality)",C15-Major Industry Group,C16-Nature of Employment (Primary Occupation)
0,1,1,2805,1,1
1,1,2,0,0,0
2,1,2,0,0,0
3,1,2,0,0,0
4,1,1,2805,1,1
...,...,...,...,...,...
394488,19,1,4712,47,0
394489,19,2,0,0,0
394490,19,2,0,0,0
394491,19,2,0,0,0


In [14]:
df_cleaned.to_csv("LFS_march2324_final.csv", index=False)